In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, time, date

FILE_PATH = 'C:\\Users\\DELL\\Downloads\\Analytics Position Case Study1.xlsx'
EXPORT_PATH = 'C:\\Users\\DELL\\Downloads\\'

def load_and_prepare_data():
    """Load and process data from Excel files"""
    
    df_gameplay = pd.read_excel(FILE_PATH, sheet_name='User Gameplay data')
    df_deposits = pd.read_excel(FILE_PATH, sheet_name='Deposit Data') 
    df_withdrawals = pd.read_excel(FILE_PATH, sheet_name='Withdrawal Data')
    
    processed_dfs = {}
    
    for df_name, df in [('gameplay', df_gameplay), ('deposits', df_deposits), ('withdrawals', df_withdrawals)]:
        
        # Standardize column names
        if 'User ID' in df.columns:
            df.rename(columns={'User ID': 'User_ID'}, inplace=True)
        if 'Games Played' in df.columns:
            df.rename(columns={'Games Played': 'Games_Played'}, inplace=True)
        if 'User Id' in df.columns:
            df.rename(columns={'User Id': 'User_ID'}, inplace=True)
        
        # Handle date format inconsistencies
        df['Datetime_orig'] = pd.to_datetime(df['Datetime'], errors='coerce')
        
        df['orig_year'] = df['Datetime_orig'].dt.year
        df['orig_month'] = df['Datetime_orig'].dt.month
        df['orig_day'] = df['Datetime_orig'].dt.day
        df['hour'] = df['Datetime_orig'].dt.hour
        df['minute'] = df['Datetime_orig'].dt.minute
        df['second'] = df['Datetime_orig'].dt.second
        
        df['Datetime_swapped'] = pd.to_datetime({
            'year': df['orig_year'],
            'month': df['orig_day'], 
            'day': df['orig_month'],
            'hour': df['hour'],
            'minute': df['minute'],
            'second': df['second']
        }, errors='coerce')
        
        # Apply date correction where needed
        final_datetimes = []
        for i in range(len(df)):
            orig_dt = df['Datetime_orig'].iloc[i]
            swap_dt = df['Datetime_swapped'].iloc[i]
            
            if pd.notna(orig_dt) and pd.notna(swap_dt):
                orig_day = orig_dt.day
                orig_month = orig_dt.month
                
                if orig_month != 10 and orig_day == 10:
                    final_datetimes.append(swap_dt)
                elif orig_month == 10:
                    final_datetimes.append(orig_dt)
                else:
                    final_datetimes.append(orig_dt)
            elif pd.notna(orig_dt):
                final_datetimes.append(orig_dt)
            elif pd.notna(swap_dt):
                final_datetimes.append(swap_dt)
            else:
                final_datetimes.append(pd.NaT)
        
        df['Datetime'] = final_datetimes
        df = df[pd.notna(df['Datetime'])].copy()
        
        # Extract time components
        df['Date'] = df['Datetime'].dt.date
        df['Hour'] = df['Datetime'].dt.hour
        df['Day'] = df['Datetime'].dt.day
        df['Month'] = df['Datetime'].dt.month
        df['Year'] = df['Datetime'].dt.year
        df['Slot'] = df['Hour'].apply(lambda x: 'S1' if x < 12 else 'S2')
        
        # Remove temporary columns
        temp_cols = ['Datetime_orig', 'Datetime_swapped', 'orig_year', 'orig_month', 'orig_day', 'hour', 'minute', 'second']
        df.drop(columns=[col for col in temp_cols if col in df.columns], inplace=True)
        
        processed_dfs[df_name] = df
    
    return processed_dfs['gameplay'], processed_dfs['deposits'], processed_dfs['withdrawals']

def calculate_loyalty_points_for_slot(gameplay_df, deposits_df, withdrawals_df, target_date, target_slot):
    """Calculate loyalty points for specific date and time slot"""
    
    gameplay_slot = gameplay_df[(gameplay_df['Date'] == target_date) & (gameplay_df['Slot'] == target_slot)]
    deposits_slot = deposits_df[(deposits_df['Date'] == target_date) & (deposits_df['Slot'] == target_slot)]
    withdrawals_slot = withdrawals_df[(withdrawals_df['Date'] == target_date) & (withdrawals_df['Slot'] == target_slot)]
    
    if len(gameplay_slot) == 0 and len(deposits_slot) == 0 and len(withdrawals_slot) == 0:
        return pd.DataFrame(columns=[
            'User_ID', 'Games_Played', 'Deposit_Amount', 'Withdrawal_Amount',
            'Deposit_Count', 'Withdrawal_Count', 'Deposit_Points', 'Withdrawal_Points',
            'Transaction_Diff_Points', 'Games_Points', 'Total_Loyalty_Points'
        ])
    
    # Aggregate data by user
    games_by_user = gameplay_slot.groupby('User_ID')['Games_Played'].sum() if len(gameplay_slot) > 0 else pd.Series(dtype=int)
    
    if len(deposits_slot) > 0:
        deposits_by_user = deposits_slot.groupby('User_ID').agg({
            'Amount': 'sum',
            'User_ID': 'count'
        }).rename(columns={'User_ID': 'deposit_count'})
    else:
        deposits_by_user = pd.DataFrame(columns=['Amount', 'deposit_count'])
    
    if len(withdrawals_slot) > 0:
        withdrawals_by_user = withdrawals_slot.groupby('User_ID').agg({
            'Amount': 'sum', 
            'User_ID': 'count'
        }).rename(columns={'User_ID': 'withdrawal_count'})
    else:
        withdrawals_by_user = pd.DataFrame(columns=['Amount', 'withdrawal_count'])
    
    # Get all unique users
    all_users = set()
    all_users.update(games_by_user.index.tolist())
    all_users.update(deposits_by_user.index.tolist())
    all_users.update(withdrawals_by_user.index.tolist())
    
    # Calculate loyalty points for each user
    results = []
    for user_id in all_users:
        games = games_by_user.get(user_id, 0)
        
        if user_id in deposits_by_user.index:
            deposit_amount = deposits_by_user.loc[user_id, 'Amount']
            deposit_count = deposits_by_user.loc[user_id, 'deposit_count']
        else:
            deposit_amount = 0
            deposit_count = 0
        
        if user_id in withdrawals_by_user.index:
            withdrawal_amount = withdrawals_by_user.loc[user_id, 'Amount']
            withdrawal_count = withdrawals_by_user.loc[user_id, 'withdrawal_count']
        else:
            withdrawal_amount = 0
            withdrawal_count = 0
        
        # Apply loyalty formula
        deposit_points = 0.01 * deposit_amount
        withdrawal_points = 0.005 * withdrawal_amount
        transaction_diff_points = 0.001 * max(deposit_count - withdrawal_count, 0)
        games_points = 0.2 * games
        total_points = deposit_points + withdrawal_points + transaction_diff_points + games_points
        
        results.append({
            'User_ID': user_id,
            'Games_Played': games,
            'Deposit_Amount': deposit_amount,
            'Withdrawal_Amount': withdrawal_amount,
            'Deposit_Count': deposit_count,
            'Withdrawal_Count': withdrawal_count,
            'Deposit_Points': deposit_points,
            'Withdrawal_Points': withdrawal_points,
            'Transaction_Diff_Points': transaction_diff_points,
            'Games_Points': games_points,
            'Total_Loyalty_Points': total_points
        })
    
    return pd.DataFrame(results).sort_values('Total_Loyalty_Points', ascending=False)

def calculate_overall_loyalty_points(gameplay_df, deposits_df, withdrawals_df, target_month=10):
    """Calculate overall monthly loyalty points and rankings"""
    
    gameplay_month = gameplay_df[gameplay_df['Month'] == target_month]
    deposits_month = deposits_df[deposits_df['Month'] == target_month]
    withdrawals_month = withdrawals_df[withdrawals_df['Month'] == target_month]
    
    games_by_user = gameplay_month.groupby('User_ID')['Games_Played'].sum()
    
    deposits_by_user = deposits_month.groupby('User_ID').agg({
        'Amount': 'sum',
        'User_ID': 'count'
    }).rename(columns={'User_ID': 'deposit_count'})
    
    withdrawals_by_user = withdrawals_month.groupby('User_ID').agg({
        'Amount': 'sum',
        'User_ID': 'count'
    }).rename(columns={'User_ID': 'withdrawal_count'})
    
    all_users = set()
    all_users.update(games_by_user.index)
    all_users.update(deposits_by_user.index)
    all_users.update(withdrawals_by_user.index)
    
    results = []
    for user_id in all_users:
        games = games_by_user.get(user_id, 0)
        deposit_amount = deposits_by_user.loc[user_id, 'Amount'] if user_id in deposits_by_user.index else 0
        withdrawal_amount = withdrawals_by_user.loc[user_id, 'Amount'] if user_id in withdrawals_by_user.index else 0
        deposit_count = deposits_by_user.loc[user_id, 'deposit_count'] if user_id in deposits_by_user.index else 0
        withdrawal_count = withdrawals_by_user.loc[user_id, 'withdrawal_count'] if user_id in withdrawals_by_user.index else 0
        
        deposit_points = 0.01 * deposit_amount
        withdrawal_points = 0.005 * withdrawal_amount  
        transaction_diff_points = 0.001 * max(deposit_count - withdrawal_count, 0)
        games_points = 0.2 * games
        total_points = deposit_points + withdrawal_points + transaction_diff_points + games_points
        
        results.append({
            'User_ID': user_id,
            'Games_Played': games,
            'Deposit_Amount': deposit_amount,
            'Withdrawal_Amount': withdrawal_amount,
            'Deposit_Count': deposit_count,
            'Withdrawal_Count': withdrawal_count,
            'Total_Loyalty_Points': total_points
        })
    
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(['Total_Loyalty_Points', 'Games_Played'], ascending=[False, False])
    df_results['Rank'] = range(1, len(df_results) + 1)
    return df_results

def analyze_statistics(gameplay_df, deposits_df, withdrawals_df, target_month=10):
    """Calculate key statistics for the analysis"""
    
    deposits_month = deposits_df[deposits_df['Month'] == target_month]
    gameplay_month = gameplay_df[gameplay_df['Month'] == target_month]
    withdrawals_month = withdrawals_df[withdrawals_df['Month'] == target_month]
    
    avg_deposit_amount = deposits_month['Amount'].mean()
    avg_deposit_per_user = deposits_month.groupby('User_ID')['Amount'].sum().mean()
    avg_games_per_user = gameplay_month.groupby('User_ID')['Games_Played'].sum().mean()
    
    return {
        'avg_deposit_amount': avg_deposit_amount,
        'avg_deposit_per_user': avg_deposit_per_user,
        'avg_games_per_user': avg_games_per_user,
        'total_users': len(set(list(gameplay_month['User_ID']) + list(deposits_month['User_ID']) + list(withdrawals_month['User_ID']))),
        'total_deposits': deposits_month['Amount'].sum(),
        'total_games': gameplay_month['Games_Played'].sum(),
        'total_withdrawals': withdrawals_month['Amount'].sum()
    }

def bonus_allocation_strategies(top_50_df, total_bonus=50000):
    """Calculate different bonus allocation strategies"""
    
    strategies = {}
    
    # Strategy 1: Proportional allocation
    total_points = top_50_df['Total_Loyalty_Points'].sum()
    strategies['proportional'] = (top_50_df['Total_Loyalty_Points'] / total_points) * total_bonus
    
    # Strategy 2: Tiered allocation
    tier_bonuses = []
    for i, row in top_50_df.iterrows():
        rank = row['Rank']
        if rank <= 10:
            tier1_total = total_bonus * 0.5
            tier1_points = top_50_df[top_50_df['Rank'] <= 10]['Total_Loyalty_Points'].sum()
            bonus = (row['Total_Loyalty_Points'] / tier1_points) * tier1_total
        elif rank <= 25:
            tier2_total = total_bonus * 0.3
            tier2_points = top_50_df[(top_50_df['Rank'] > 10) & (top_50_df['Rank'] <= 25)]['Total_Loyalty_Points'].sum()
            bonus = (row['Total_Loyalty_Points'] / tier2_points) * tier2_total
        else:
            tier3_total = total_bonus * 0.2
            tier3_points = top_50_df[top_50_df['Rank'] > 25]['Total_Loyalty_Points'].sum()
            bonus = (row['Total_Loyalty_Points'] / tier3_points) * tier3_total
        
        tier_bonuses.append(bonus)
    
    strategies['tiered'] = pd.Series(tier_bonuses, index=top_50_df.index)
    
    # Strategy 3: Hybrid approach
    points_norm = top_50_df['Total_Loyalty_Points'] / top_50_df['Total_Loyalty_Points'].max()
    games_norm = top_50_df['Games_Played'] / max(top_50_df['Games_Played'].max(), 1)
    deposits_norm = top_50_df['Deposit_Amount'] / max(top_50_df['Deposit_Amount'].max(), 1)
    
    hybrid_score = (0.5 * points_norm + 0.3 * games_norm + 0.2 * deposits_norm)
    total_hybrid = hybrid_score.sum()
    strategies['hybrid'] = (hybrid_score / total_hybrid) * total_bonus
    
    return strategies

def export_all_results_to_excel(gameplay_df, deposits_df, withdrawals_df, output_filename='ABC_Gaming_Analysis_Complete.xlsx'):
    """Export all analysis results to Excel file"""
    
    gameplay_clean, deposits_clean, withdrawals_clean = load_and_prepare_data()
    
    output_path = EXPORT_PATH + output_filename
    
    overall_loyalty = calculate_overall_loyalty_points(
        gameplay_clean, deposits_clean, withdrawals_clean, target_month=10
    )
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        
        # Part A: User Rankings and Slot Analysis
        overall_loyalty.to_excel(writer, sheet_name='PartA_Overall_Rankings', index=False)
        
        slot_analyses = {
            '2nd_Oct_S1': (date(2022, 10, 2), 'S1'),
            '16th_Oct_S2': (date(2022, 10, 16), 'S2'),
            '18th_Oct_S1': (date(2022, 10, 18), 'S1'),
            '26th_Oct_S2': (date(2022, 10, 26), 'S2')
        }
        
        print("Processing slot analyses...")
        slot_results = {}
        
        for slot_name, (target_date, slot) in slot_analyses.items():
            print(f"Processing {slot_name}...")
            result = calculate_loyalty_points_for_slot(
                gameplay_clean, deposits_clean, withdrawals_clean, target_date, slot
            )
            slot_results[slot_name] = result
            
            sheet_name = f"PartA_{slot_name}"
            result.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Completed {slot_name}: {len(result)} users analyzed")
        
        # Statistical analysis
        stats = analyze_statistics(gameplay_clean, deposits_clean, withdrawals_clean, target_month=10)
        
        stats_df = pd.DataFrame([
            ['Analysis Period', 'October 2022'],
            ['Data Status', 'Complete'],
            ['October 2nd Data', 'Available'],
            ['October 16th Data', 'Available'],
            ['October 18th Data', 'Available'],
            ['October 26th Data', 'Available'],
            ['', ''],
            ['Average Deposit Amount', f"Rs{stats['avg_deposit_amount']:,.2f}"],
            ['Average Deposit per User', f"Rs{stats['avg_deposit_per_user']:,.2f}"],
            ['Average Games per User', f"{stats['avg_games_per_user']:.2f}"],
            ['Total Active Users', f"{stats['total_users']:,}"],
            ['Total Deposits (October)', f"Rs{stats['total_deposits']:,}"],
            ['Total Games Played (October)', f"{stats['total_games']:,}"],
            ['Total Withdrawals', f"Rs{stats['total_withdrawals']:,}"],
        ], columns=['Metric', 'Value'])
        
        stats_df.to_excel(writer, sheet_name='PartA_Statistics', index=False)
        
        # Part B: Bonus Allocation
        print("Processing bonus allocation strategies...")
        
        top_50 = overall_loyalty.head(50)
        strategies = bonus_allocation_strategies(top_50, total_bonus=50000)
        
        bonus_comparison = top_50[['User_ID', 'Rank', 'Total_Loyalty_Points', 'Games_Played', 
                                  'Deposit_Amount', 'Withdrawal_Amount']].copy()
        bonus_comparison['Strategy1_Proportional'] = strategies['proportional'].values
        bonus_comparison['Strategy2_Tiered'] = strategies['tiered'].values
        bonus_comparison['Strategy3_Hybrid'] = strategies['hybrid'].values
        
        bonus_comparison['Prop_Percentage'] = (bonus_comparison['Strategy1_Proportional'] / 50000 * 100).round(2)
        bonus_comparison['Tiered_Percentage'] = (bonus_comparison['Strategy2_Tiered'] / 50000 * 100).round(2)
        bonus_comparison['Hybrid_Percentage'] = (bonus_comparison['Strategy3_Hybrid'] / 50000 * 100).round(2)
        
        bonus_comparison.to_excel(writer, sheet_name='PartB_Bonus_Allocation', index=False)
        
        strategy_summary = pd.DataFrame([
            ['Strategy', 'Description', 'Top 5 Share (%)', 'Top 10 Share (%)', 'Assessment'],
            ['Proportional', 'Based purely on loyalty points', 
             f"{bonus_comparison['Strategy1_Proportional'].head(5).sum()/50000*100:.1f}%",
             f"{bonus_comparison['Strategy1_Proportional'].head(10).sum()/50000*100:.1f}%",
             'Concentrates rewards on top performers'],
            ['Tiered', 'Tier 1: 50%, Tier 2: 30%, Tier 3: 20%',
             f"{bonus_comparison['Strategy2_Tiered'].head(5).sum()/50000*100:.1f}%", 
             f"{bonus_comparison['Strategy2_Tiered'].head(10).sum()/50000*100:.1f}%",
             'Balanced distribution across tiers'],
            ['Hybrid', 'Points(50%) + Games(30%) + Deposits(20%)',
             f"{bonus_comparison['Strategy3_Hybrid'].head(5).sum()/50000*100:.1f}%",
             f"{bonus_comparison['Strategy3_Hybrid'].head(10).sum()/50000*100:.1f}%", 
             'Multi-factor approach']
        ])
        
        strategy_summary.to_excel(writer, sheet_name='PartB_Strategy_Comparison', index=False, header=False)
        
        # Part C: Formula Analysis
        print("Processing formula analysis...")
        
        formula_analysis = overall_loyalty.copy()
        formula_analysis['Deposit_Points'] = formula_analysis['Deposit_Amount'] * 0.01
        formula_analysis['Withdrawal_Points'] = formula_analysis['Withdrawal_Amount'] * 0.005
        formula_analysis['Transaction_Diff_Points'] = np.maximum(
            formula_analysis['Deposit_Count'] - formula_analysis['Withdrawal_Count'], 0) * 0.001
        formula_analysis['Games_Points'] = formula_analysis['Games_Played'] * 0.2
        
        formula_analysis['Deposit_Contribution_%'] = (
            formula_analysis['Deposit_Points'] / formula_analysis['Total_Loyalty_Points'] * 100).fillna(0)
        formula_analysis['Withdrawal_Contribution_%'] = (
            formula_analysis['Withdrawal_Points'] / formula_analysis['Total_Loyalty_Points'] * 100).fillna(0)
        formula_analysis['Games_Contribution_%'] = (
            formula_analysis['Games_Points'] / formula_analysis['Total_Loyalty_Points'] * 100).fillna(0)
        formula_analysis['Transaction_Diff_Contribution_%'] = (
            formula_analysis['Transaction_Diff_Points'] / formula_analysis['Total_Loyalty_Points'] * 100).fillna(0)
        
        formula_export = formula_analysis[['User_ID', 'Rank', 'Total_Loyalty_Points', 
                                          'Deposit_Points', 'Withdrawal_Points', 'Games_Points',
                                          'Transaction_Diff_Points', 'Deposit_Contribution_%',
                                          'Withdrawal_Contribution_%', 'Games_Contribution_%',
                                          'Transaction_Diff_Contribution_%']].head(100)
        
        formula_export.to_excel(writer, sheet_name='PartC_Formula_Analysis', index=False)
        
        fairness_issues = pd.DataFrame([
            ['Issue', 'Current Impact', 'Problem Description', 'Suggested Solution'],
            ['Deposit Dominance', f"{formula_analysis['Deposit_Contribution_%'].mean():.1f}% avg contribution", 
             'High deposits override gaming skill', 'Reduce multiplier and add caps'],
            ['Withdrawal Rewards', f"{formula_analysis['Withdrawal_Contribution_%'].mean():.1f}% avg contribution",
             'Rewards money leaving platform', 'Remove withdrawal points'],
            ['Low Gaming Weight', f"{formula_analysis['Games_Contribution_%'].mean():.1f}% avg contribution",
             'Core activity undervalued', 'Increase games multiplier'],
            ['Missing Skill Metrics', '0% contribution',
             'No performance consideration', 'Add win rate bonuses'],
            ['No Time Decay', '100% permanent value',
             'Old activities weighted equally', 'Apply decay factors'],
            ['Gaming Potential', f"{formula_analysis['Transaction_Diff_Contribution_%'].mean():.1f}% avg impact",
             'Can be manipulated', 'Add minimum thresholds']
        ])
        
        fairness_issues.to_excel(writer, sheet_name='PartC_Fairness_Assessment', index=False, header=False)
        
        improved_formula = pd.DataFrame([
            ['Component', 'Current Multiplier', 'Suggested Multiplier', 'Rationale'],
            ['Games Played', '0.2', '1.0', 'Make gaming primary factor'],
            ['Win Rate Bonus', 'N/A', '+0.5 per game if >60% wins', 'Reward skill'],
            ['Deposits', '0.01', '0.005 (capped)', 'Reduce pay-to-win'],
            ['Withdrawals', '0.005', '0.000', 'Remove conflicting incentive'],
            ['Consistency', 'N/A', '+0.5 per active day', 'Reward engagement'],
            ['Recent Activity', 'N/A', '1.2x multiplier', 'Emphasize current activity'],
            ['Streak Bonus', 'N/A', '+10% per consecutive day', 'Reward sustained play'],
            ['Account Age', 'N/A', '+5% per month', 'Reward loyalty']
        ])
        
        improved_formula.to_excel(writer, sheet_name='PartC_Improved_Formula', index=False, header=False)
        
        # Summary
        executive_summary = pd.DataFrame([
            ['ABC GAMING LOYALTY ANALYSIS SUMMARY', ''],
            ['', ''],
            ['Analysis Period', 'October 2022'],
            ['Required Dates Analyzed', '2nd, 16th, 18th, 26th October 2022'],
            ['Data Processing Status', 'Complete'],
            ['', ''],
            ['PART A RESULTS', ''],
            ['Total Users Analyzed', f"{len(overall_loyalty):,}"],
            ['Average Loyalty Points', f"{overall_loyalty['Total_Loyalty_Points'].mean():.2f}"],
            ['Top Player Points', f"{overall_loyalty['Total_Loyalty_Points'].iloc[0]:.2f}"],
            ['October 2nd S1 Users', f"{len(slot_results['2nd_Oct_S1'])}"],
            ['October 16th S2 Users', f"{len(slot_results['16th_Oct_S2'])}"],
            ['October 18th S1 Users', f"{len(slot_results['18th_Oct_S1'])}"],
            ['October 26th S2 Users', f"{len(slot_results['26th_Oct_S2'])}"],
            ['', ''],
            ['PART B RESULTS', ''],
            ['Recommended Strategy', 'Tiered System'],
            ['Tier 1 Allocation', '50% of Rs 50,000'],
            ['Tier 2 Allocation', '30% of Rs 50,000'],
            ['Tier 3 Allocation', '20% of Rs 50,000'],
            ['', ''],
            ['PART C ASSESSMENT', ''],
            ['Formula Fairness', 'Needs improvement'],
            ['Main Issue', 'Deposit bias over gaming merit'],
            ['Primary Recommendation', 'Rebalance toward gameplay']
        ], columns=['Item', 'Value'])
        
        executive_summary.to_excel(writer, sheet_name='Executive_Summary', index=False, header=False)
    
    print(f"\nAnalysis complete. Results saved to: {output_path}")
    print(f"Part A: User rankings and slot analyses completed")
    print(f"Part B: Bonus allocation strategies completed") 
    print(f"Part C: Formula analysis and recommendations completed")
    
    return overall_loyalty, bonus_comparison, stats, slot_results

# Execute analysis
try:
    print("Starting ABC Gaming Loyalty Analysis")
    print("Processing Parts A, B, and C")
    
    df_gameplay = pd.read_excel(FILE_PATH, sheet_name='User Gameplay data')
    df_deposits = pd.read_excel(FILE_PATH, sheet_name='Deposit Data') 
    df_withdrawals = pd.read_excel(FILE_PATH, sheet_name='Withdrawal Data')
    
    results, bonus_df, statistics, slots = export_all_results_to_excel(
        df_gameplay, df_deposits, df_withdrawals
    )
    
    print("\nAnalysis completed successfully")
    print("All assignment requirements have been fulfilled")
    
except Exception as e:
    print(f"Error occurred: {e}")
    import traceback
    traceback.print_exc()


Starting ABC Gaming Loyalty Analysis
Processing Parts A, B, and C
Processing slot analyses...
Processing 2nd_Oct_S1...
Completed 2nd_Oct_S1: 636 users analyzed
Processing 16th_Oct_S2...
Completed 16th_Oct_S2: 594 users analyzed
Processing 18th_Oct_S1...
Completed 18th_Oct_S1: 622 users analyzed
Processing 26th_Oct_S2...
Completed 26th_Oct_S2: 628 users analyzed
Processing bonus allocation strategies...
Processing formula analysis...

Analysis complete. Results saved to: C:\Users\DELL\Downloads\ABC_Gaming_Analysis_Complete.xlsx
Part A: User rankings and slot analyses completed
Part B: Bonus allocation strategies completed
Part C: Formula analysis and recommendations completed

Analysis completed successfully
All assignment requirements have been fulfilled
